In [4]:
import tensorflow as tf
import numpy as np
from keras.utils import to_categorical
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout
from keras.applications.vgg16 import VGG16
from keras import Sequential

########
# data loading
########

img_height=128
img_width=128

data_dir='/content/drive/My Drive/Dati/ann-and-dl-image-classification/Classification_Dataset/training'
    # qui metti l'indirizzo della cartella contenente il training set, così
    # come la scarichi dal sito del prof

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  seed=123,
  subset="training",
  label_mode='categorical',
  image_size=(img_height, img_width))

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  seed=123,
  subset="validation",
  label_mode='categorical',
  image_size=(img_height, img_width))

Found 1554 files belonging to 20 classes.
Using 1244 files for training.
Found 1554 files belonging to 20 classes.
Using 310 files for validation.


In [3]:
n=20

### tranfert learning:
### prendi il modello già fatto e setti i pesi come non allenabili

vgg=VGG16(input_shape=[img_height,img_width,3],weights='imagenet',include_top=False)

for l in vgg.layers:
  l.trainable=False




model=Sequential()

### image agumentation:
### mettiamo degli strati iniziali che ribaltano e ruotano l'input a caso

model.add(tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal',input_shape=[img_height,img_width,3]))
model.add(tf.keras.layers.experimental.preprocessing.RandomRotation(0.2))


model.add(vgg)
model.add(Flatten())
model.add(Dense(n,activation='softmax'))

model.summary()


model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(train_ds,epochs=5,validation_data=val_ds,batch_size=16)
### overfitta

58892288/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip (RandomFlip)     (None, 128, 128, 3)       0         
_________________________________________________________________
random_rotation (RandomRotat (None, 128, 128, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 20)                163860    
Total params: 14,878,548
Trainable params: 163,860
Non-trainable params: 14,714,688
_________________________________________________________________
